# Concepts

### Kubernetes Components
* <b>Master Components</b>: Master components provide the cluster’s control plane. Master components make global decisions about the cluster (for example, scheduling), and detecting and responding to cluster events (starting up a new pod when a replication controller’s ‘replicas’ field is unsatisfied).
    * `kube-apiserver`: Component on the master that exposes the Kubernetes API. It is the front-end for the Kubernetes control plane. It is designed to scale horizontally – that is, it scales by deploying more instances. 
    * `etcd`: Consistent and highly-available key value store used as Kubernetes’ backing store for all cluster data.
    * `kube-scheduler`: Component on the master that watches newly created pods that have no node assigned, and selects a node for them to run on.
    * `kube-controller-manager`: Component on the master that runs controllers. (Logically, each controller is a separate process, but to reduce complexity, they are all compiled into a single binary and run in a single process.)
        * Node Controller: Responsible for noticing and responding when nodes go down.
        * Replication Controller: Responsible for maintaining the correct number of pods for every replication controller object in the system.
        * Endpoints Controller: Populates the Endpoints object (that is, joins Services & Pods).
        * Service Account & Token Controllers: Create default accounts and API access tokens for new namespaces.
    * `cloud-controller-manager`: cloud-controller-manager runs controllers that interact with the underlying cloud providers. cloud-controller-manager runs cloud-provider-specific controller loops only. You must disable these controller loops in the kube-controller-manager. 
        * Node Controller: For checking the cloud provider to determine if a node has been deleted in the cloud after it stops responding
        * Route Controller: For setting up routes in the underlying cloud infrastructure
        * Service Controller: For creating, updating and deleting cloud provider load balancers
        * Volume Controller: For creating, attaching, and mounting volumes, and interacting with the cloud provider to orchestrate volumes
* <b>Node Components</b>: Node components run on every node, maintaining running pods and providing the Kubernetes runtime environment.
    * `kubelet`: An agent that runs on each node in the cluster. It makes sure that containers are running in a pod. The kubelet takes a set of PodSpecs that are provided through various mechanisms and ensures that the containers described in those PodSpecs are running and healthy.
    * `kube-proxy`: kube-proxy enables the Kubernetes service abstraction by maintaining network rules on the host and performing connection forwarding.
    * `Container Runtime`: The container runtime is the software that is responsible for running containers. Kubernetes supports several runtimes: `Docker, rkt, runc` and` any OCI runtime-spec` implementation.
* <b>Addons</b>: Addons are pods and services that implement cluster features. The pods may be managed by Deployments, ReplicationControllers, and so on. https://kubernetes.io/docs/concepts/cluster-administration/addons/
    * `Cluster DNS` is a DNS server, in addition to the other DNS server(s) in your environment, which serves DNS records for Kubernetes services. Containers started by Kubernetes automatically include this DNS server in their DNS searches.
    * `Web UI (Dashboard)`: Dashboard is a general purpose, web-based UI for Kubernetes clusters. It allows users to manage and troubleshoot applications running in the cluster, as well as the cluster itself. https://kubernetes.io/docs/tasks/access-application-cluster/web-ui-dashboard/
    * `Container Resource Monitoring`: Container Resource Monitoring records generic time-series metrics about containers in a central database, and provides a UI for browsing that data. https://kubernetes.io/docs/tasks/debug-application-cluster/resource-usage-monitoring/
    * `Cluster-level Logging`: A Cluster-level logging mechanism is responsible for saving container logs to a central log store with search/browsing interface. https://kubernetes.io/docs/concepts/cluster-administration/logging/

**** 

### Kubernetes Objects
* Kubernetes Objects are persistent entities in the Kubernetes system. Kubernetes uses these entities to represent the state of your cluster. 
    * <i>To work with Kubernetes objects–whether to create, modify, or delete them–you’ll need to use the `Kubernetes API`. When you use the kubectl command-line interface, the CLI makes the necessary Kubernetes API calls for you. You can also use the Kubernetes API directly in your own programs using one of the `Client Libraries`</i>(https://kubernetes.io/docs/reference/using-api/client-libraries/).
* <b>Object Spec and Status</b>: Every Kubernetes object includes two nested object fields that govern the object’s configuration: the object `spec` and the `object status`.
    * `The spec`, which you must provide, describes your desired state for the object–the characteristics that you want the object to have. 
    * `The status` describes the actual state of the object, and is supplied and updated by the Kubernetes system. 
    * <i>Describing a Kubernetes Object: When you create an object in Kubernetes, you must provide the object spec that describes its desired state, as well as some basic information about the object. When you use the Kubernetes API to create the object, that API request must include that information as JSON in the request body. Most often, you provide the information to kubectl in a .yaml file. kubectl converts the information to JSON when making the API request.</i>
* <b>Required Fields</b>: https://kubernetes.io/docs/concepts/overview/working-with-objects/kubernetes-objects/#required-fields
    * Names: A client-provided string that refers to an object in a resource URL, such as `/api/v1/pods/some-name`.
    * UIDs: A Kubernetes systems-generated string to uniquely identify objects.
    * Namespaces: Kubernetes supports multiple virtual clusters backed by the same physical cluster. These virtual clusters are called namespaces. Namespaces provide a scope for names. Names of resources need to be unique within a namespace, but not across namespaces.
    * Labels and Selectors: https://kubernetes.io/docs/concepts/overview/working-with-objects/common-labels/
        * Labels are key/value pairs that are attached to objects, such as pods. Labels can be used to select objects and to find collections of objects that satisfy certain conditions. 
            * 
            ```json
                "metadata": {
                  "labels": {
                    "key1" : "value1",
                    "key2" : "value2"
                  }
                }
            ```
            * Label selectors: Via a label selector, the client/user can identify a set of objects. 
            * The label selector is the core grouping primitive in Kubernetes. 
            * The API currently supports two types of selectors: 
                * `Equality- or inequality-based` requirements allow filtering by label keys and values. Matching objects must satisfy all of the specified label constraints. Three kinds of operators are admitted `=,==,!=`.
                * `Set-based label requirements` allow filtering keys according to a set of values. Three kinds of operators are supported: `in,notin and exists`.
        * LIST and WATCH filtering: operations may specify label selectors to filter the sets of objects returned using a query parameter. 
            * 
            ```yml
            selector:
              matchLabels:
                component: redis
              matchExpressions:
                - {key: tier, operator: In, values: [cache]}
                - {key: environment, operator: NotIn, values: [dev]}
            ```
            * https://kubernetes.io/docs/concepts/overview/working-with-objects/labels/#list-and-watch-filtering
            * One use case for selecting over labels is to constrain the set of nodes onto which a pod can schedule.
    * Annotations: use Kubernetes annotations to attach arbitrary non-identifying metadata to objects. Clients such as tools and libraries can retrieve this metadata. annotations are not used to identify and select objects. 
    * Field Selectors: Field selectors let you select Kubernetes resources based on the value of one or more resource fields. All resource types support the `metadata.name` and `metadata.namespace` fields. 
* <b>Object Management Using kubectl</b>: https://kubernetes.io/docs/concepts/overview/object-management-kubectl/overview/
    * Imperative commands: When using imperative commands, a user operates directly on live objects in a cluster. The user provides operations to the kubectl command as arguments or flags.
        * `kubectl run nginx --image nginx`
        * https://kubernetes.io/docs/concepts/overview/object-management-kubectl/imperative-command/
    * Imperative object configuration: In imperative object configuration, the kubectl command specifies the operation (create, replace, etc.), optional flags and at least one file name. The file specified must contain a full definition of the object in YAML or JSON format.
        * `kubectl create -f nginx.yaml`
        * https://kubernetes.io/docs/concepts/overview/object-management-kubectl/imperative-config/
    * Declarative object configuration: When using declarative object configuration, a user operates on object configuration files stored locally, however the user does not define the operations to be taken on the files. Create, update, and delete operations are automatically detected per-object by kubectl. 
        * `kubectl apply -f configs/`
        * https://kubernetes.io/docs/concepts/overview/object-management-kubectl/declarative-config/

****

# Kubernetes Architecture
* <b>Nodes</b>: A node is a worker machine in Kubernetes, previously known as a minion. A node may be a VM or physical machine, depending on the cluster. Each node contains the services necessary to run pods and is managed by the master components. 
    * <b>Node Status</b>: 
        * Addresses: 
            * HostName: The hostname as reported by the node’s kernel. Can be overridden via the `kubelet --hostname-override` parameter.
            * ExternalIP: Typically the IP address of the node that is externally routable (available from outside the cluster).
            * InternalIP: Typically the IP address of the node that is routable only within the cluster.
        * Condition: The conditions field describes the status of all Running nodes.
            * https://kubernetes.io/docs/concepts/architecture/nodes/#condition
        * Capacity: Describes the resources available on the node: CPU, memory and the maximum number of pods that can be scheduled onto the node.
        * Info: General information about the node, such as kernel version, Kubernetes version (kubelet and kube-proxy version), Docker version (if used), OS name. The information is gathered by Kubelet from the node.
    * <b>Management</b>: 
        * Node Controller: The node controller is a Kubernetes master component which manages various aspects of nodes. https://kubernetes.io/docs/concepts/architecture/nodes/#node-controller
            * The first is assigning a CIDR block to the node when it is registered
            * The second is keeping the node controller’s internal list of nodes up to date with the cloud provider’s list of available machines. 
            * The third is monitoring the nodes’ health. The node controller is responsible for updating the NodeReady condition of NodeStatus to ConditionUnknown when a node becomes unreachable and then later evicting all the pods from the node if the node continues to be unreachable.
        * Self-Registration of Nodes: When the kubelet flag `--register-node` is true (the default), the kubelet will attempt to register itself with the API server. 
        * Node capacity: The capacity of the node (number of cpus and amount of memory) is part of the node object. Normally, nodes register themselves and report their capacity when creating the node object. 
* <b>Master-Node communication</b>: https://kubernetes.io/docs/concepts/architecture/master-node-communication/
* <b>Cloud Controller Manager</b>: https://kubernetes.io/docs/concepts/architecture/cloud-controller/
    * The cloud controller manager runs alongside other master components such as the Kubernetes controller manager, the API server, and scheduler. It can also be started as a Kubernetes addon, in which case it runs on top of Kubernetes.
    * The cloud controller manager’s design is based on a plugin mechanism that allows new cloud providers to integrate with Kubernetes easily by using plugins.
    * <b>Architecture of a Kubernetes cluster without the cloud controller manager and with it</b>:
    <table>
        <tr>
            <td><img src="../images/devops/pre-ccm-arch.png" width="500px"></td>
            <td><img src="../images/devops/post-ccm-arch.png" width="500px"></td>
        </tr>
    </table>
    
****

## Containers: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#container-v1-core
* <b>Images</b>: You create your Docker image and push it to a registry before referring to it in a Kubernetes pod. The image property of a container supports the same syntax as the docker command does, including private registries and tags. https://kubernetes.io/docs/concepts/containers/images/
* <b>Container Environment Variables</b>: 
    * Container information: 
        * The hostname of a Container is the name of the Pod in which the Container is running.
        * User defined environment variables from the Pod definition are also available to the Container, as are any environment variables specified statically in the Docker image.
    * Cluster information: A list of all services that were running when a Container was created is available to that Container as environment variables.
* <b>Runtime Class</b>: RuntimeClass is an alpha feature for selecting the container runtime configuration to use to run a pod’s containers.
* <b>Container Lifecycle Hooks</b>: kubelet managed Containers can use the Container lifecycle hook framework to run code triggered by events during their management lifecycle.
    * `PostStart`: This hook executes immediately after a container is created.
    * `PreStop`: This hook is called immediately before a container is terminated.
    
****

## Pods: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#pod-v1-core
* <b>Pod Overview</b>: A Pod is the basic building block of Kubernetes–the smallest and simplest unit in the Kubernetes object model that you create or deploy. A Pod represents a running process on your cluster.
    * A pod models an application-specific “logical host” - it contains one or more application containers which are relatively tightly coupled — in a pre-container world, being executed on the same physical or virtual machine would mean being executed on the same logical host.
    * A Pod encapsulates an application container storage resources, a unique network IP, and options that govern how the container(s) should run. 
    * A Pod represents a unit of deployment: a single instance of an application in Kubernetes, which might consist of either a single container or a small number of containers that are tightly coupled and that share resources.
        * <b>Pods that run a single container</b>: The “one-container-per-Pod” model is the most common Kubernetes use case; in this case, you can think of a Pod as a wrapper around a single container, and Kubernetes manages the Pods rather than the containers directly.
        * <b>Pods that run multiple containers</b>: A Pod might encapsulate an application composed of multiple co-located containers that are tightly coupled and need to share resources. The Pod wraps these containers and storage resources together as a single manageable entity.
            * The containers in a Pod are automatically co-located and co-scheduled on the same physical or virtual machine in the cluster. 
            * The containers can share resources and dependencies, communicate with one another, and coordinate when and how they are terminated. Pods provide two kinds of shared resources for their constituent containers:
                * <b>Networking</b>: Each Pod is assigned a unique IP address. Every container in a Pod shares the network namespace, including the IP address and network ports. Containers inside a Pod can communicate with one another using localhost. When containers in a Pod communicate with entities outside the Pod, they must coordinate how they use the shared network resources (such as ports).
                * <b>Storage</b>: A Pod can specify a set of shared storage volumes. All containers in the Pod can access the shared volumes, allowing those containers to share data. Volumes also allow persistent data in a Pod to survive in case one of the containers within needs to be restarted. 
            * <img src="../images/devops/pod.svg" width="200px">
    * Each Pod is meant to run a single instance of a given application. If you want to scale your application horizontally, you should use multiple Pods, one for each instance. <i>Replicated Pods are usually created and managed as a group by an abstraction called a Controller</i>.
    * <i>The hostname is set to the pod’s Name for the application containers within the pod</i>.

* Pods runtime behaivour:
    * <i>When a Pod gets created, it is scheduled to run on a Node in your cluster. The Pod remains on that Node until the process is terminated, the pod object is deleted, the pod is evicted for lack of resources, or the Node fails.</i>
    * <i>Pods do not self-heal. If a Pod is scheduled to a Node that fails, or if the scheduling operation itself fails, the Pod is deleted; likewise, a Pod won’t survive an eviction due to a lack of resources or Node maintenance.</i>
    * <i>Restarting a container in a Pod should not be confused with restarting the Pod. The Pod itself does not run, but is an environment the containers run in and persists until it is deleted.</i>
* <b>Pods and Controllers</b>: A Controller can create and manage multiple Pods for you, handling replication and rollout and providing self-healing capabilities at cluster scope.
    * <b>Pod Templates</b>: Pod templates are pod specifications which are included in other objects, such as `Replication Controllers, Jobs, and DaemonSets`. Controllers use Pod Templates to make actual pods. 
    * <b>Pod advantages</b>:
        * scheduler and controller pluggability
        * support for pod-level operations without the need to “proxy” them via controller APIs
        * decoupling of pod lifetime from controller lifetime, such as for bootstrapping
        * decoupling of controllers and services — the endpoint controller just watches pods
        * clean composition of Kubelet-level functionality with cluster-level functionality — Kubelet is effectively the “pod controller”
        * high-availability applications, which will expect pods to be replaced in advance of their termination and certainly in advance of deletion, such as in the case of planned evictions or image prefetching.
* <b>Pod Lifecycle</b>:
    * <b>Termination of Pods</b>: 
        1. User sends command to delete Pod, with default grace period (30s)
        2. The Pod in the API server is updated with the time beyond which the Pod is considered “dead” along with the grace period.
        3. Pod shows up as “Terminating” when listed in client commands
        4. (simultaneous with 3) When the Kubelet sees that a Pod has been marked as terminating because the time in 2 has been set, it begins the pod shutdown process.
            * If one of the Pod’s containers has defined a preStop hook, it is invoked inside of the container. If the preStop hook is still running after the grace period expires, step 2 is then invoked with a small (2 second) extended grace period.
            * The container is sent the TERM signal. Note that not all containers in the Pod will receive the TERM signal at the same time and may each require a preStop hook if the order in which they shut down matters.
        5. (simultaneous with 3) Pod is removed from endpoints list for service, and are no longer considered part of the set of running pods for replication controllers. Pods that shutdown slowly cannot continue to serve traffic as load balancers (like the service proxy) remove them from their rotations.
        6. When the grace period expires, any processes still running in the Pod are killed with SIGKILL.
        7. The Kubelet will finish deleting the Pod on the API server by setting grace period 0 (immediate deletion). The Pod disappears from the API and is no longer visible from the client.
    * Pod phase, Pod conditions, Container probes, Pod readiness gate, Restart policy, Pod lifetime
        * https://kubernetes.io/docs/concepts/workloads/pods/pod-lifecycle/
* <b>Init Containers</b>: Init Containers, which are specialized Containers that run before app Containers and can contain utilities or setup scripts not present in an app image.
    * https://kubernetes.io/docs/concepts/workloads/pods/init-containers/
* <b>Pod Preset</b>: A Pod Preset is an API resource for injecting additional runtime requirements into a Pod at creation time. It works as:
    * Retrieve all PodPresets available for use.
    * Check if the label selectors of any PodPreset matches the labels on the pod being created.
    * Attempt to merge the various resources defined by the PodPreset into the Pod being created.
    * On error, throw an event documenting the merge error on the pod, and create the pod without any injected resources from the PodPreset.
    * Annotate the resulting modified Pod spec to indicate that it has been modified by a PodPreset. The annotation is of the form `podpreset.admission.kubernetes.io/podpreset-<pod-preset name>: "<resource version>"`.
* <b>Disruptions solutions</b>: https://kubernetes.io/docs/concepts/workloads/pods/disruptions/
    * An Application Owner can create a PodDisruptionBudget object (PDB) for each application. A PDB limits the number of pods of a replicated application that are down simultaneously from voluntary disruptions. 
        
****

## Controllers
* <b>ReplicaSet</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#replicaset-v1-apps
    * https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/
    * ReplicaSet is the next-generation Replication Controller. The only difference between a ReplicaSet and a Replication Controller right now is the selector support. ReplicaSet supports the new set-based selector requirements as described in the labels user guide whereas a Replication Controller only supports equality-based selector requirements.
    * <b>ReplicaSets can be used independently, today it’s mainly used by Deployments as a mechanism to orchestrate pod creation, deletion and updates.</b>
    * When you use Deployments you don’t have to worry about managing the ReplicaSets that they create. Deployments own and manage their ReplicaSets.
    * <b>Alternatives to ReplicaSet: Deployment, Bare Pods, Job, DaemonSet</b>
* <b>ReplicationController</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#replicationcontroller-v1-core
    * https://kubernetes.io/docs/concepts/workloads/controllers/replicationcontroller/
    * A ReplicationController ensures that a specified number of pod replicas are running at any one time. In other words, a ReplicationController makes sure that a pod or a homogeneous set of pods is always up and available.
    * the pods maintained by a ReplicationController are automatically replaced if they fail, are deleted, or are terminated. 
    * <b>Replication controller is a top-level resource in the Kubernetes REST API. Need extra command working with Updating and Rolling back operation for pod. `kubectl rolling-update`</b>
    * <i>A Deployment that configures a ReplicaSet is now the recommended way to set up replication.</i>
* <b>Deployments</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#deployment-v1-apps
    * https://kubernetes.io/docs/concepts/workloads/controllers/deployment/
    * A Deployment controller provides declarative updates for Pods and ReplicaSets. <b>Will create ReplicaSets automaticly, contains Updating and Rolling back pod operation by default. `kubectl rollout`</b>
    * You describe a desired state in a Deployment object, and the Deployment controller changes the actual state to the desired state at a controlled rate. 
    * <b>Alternative to Deployments</b>:
        * `kubectl rolling update` updates Pods and `ReplicationControllers` in a similar fashion. But Deployments are recommended, since they are declarative, server side, and have additional features, such as rolling back to any previous revision even after the rolling update is done.
* <b>StatefulSets</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#statefulset-v1-apps
    * https://kubernetes.io/docs/concepts/workloads/controllers/statefulset/
    * StatefulSet is the workload API object used to manage stateful applications.
    * Manages the deployment and scaling of a set of Pods, and provides guarantees about the ordering and uniqueness of these Pods.
    * Like a Deployment , a StatefulSet manages Pods that are based on an identical container spec. Unlike a Deployment, a StatefulSet maintains a sticky identity for each of their Pods. These pods are created from the same spec, but are not interchangeable: each has a persistent identifier that it maintains across any rescheduling.
    * A StatefulSet operates under the same pattern as any other Controller. You define your desired state in a StatefulSet object, and the StatefulSet controller makes any necessary updates to get there from the current state.
    * <b>Stable Network ID</b>: https://kubernetes.io/docs/concepts/workloads/controllers/statefulset/#stable-network-id
    * <b>Deployment and Scaling Guarantees</b>: https://kubernetes.io/docs/concepts/workloads/controllers/statefulset/#deployment-and-scaling-guarantees
        * OrderedReady Pod Management:
            * For a StatefulSet with N replicas, when Pods are being deployed, they are created sequentially, in order from {0..N-1}.
            * When Pods are being deleted, they are terminated in reverse order, from {N-1..0}.
            * Before a scaling operation is applied to a Pod, all of its predecessors must be Running and Ready.
            * Before a Pod is terminated, all of its successors must be completely shutdown.
        * Parallel Pod Management: Parallel pod management tells the StatefulSet controller to launch or terminate all Pods in parallel, and to not wait for Pods to become Running and Ready or completely terminated prior to launching or terminating another Pod.
* <b>DaemonSet</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#daemonset-v1-apps
    * https://kubernetes.io/docs/concepts/workloads/controllers/daemonset/
    * A DaemonSet ensures that all (or some) Nodes run a copy of a Pod. As nodes are added to the cluster, Pods are added to them. As nodes are removed from the cluster, those Pods are garbage collected. Deleting a DaemonSet will clean up the Pods it created.
    * Some typical uses of a DaemonSet are:
        * running a cluster storage daemon, such as glusterd, ceph, on each node.
        * running a logs collection daemon on every node, such as fluentd or logstash.
        * running a node monitoring daemon on every node, such as Prometheus Node Exporter, collectd, Dynatrace OneAgent, AppDynamics Agent, Datadog agent, New Relic agent, Ganglia gmond or Instana agent.
    * <b>Alternatives to DaemonSet: Init Scripts, Bare Pods, Static Pods, Deployments</b>
        * https://kubernetes.io/docs/concepts/workloads/controllers/daemonset/#alternatives-to-daemonset
* <b>Jobs</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#job-v1-batch
    * https://kubernetes.io/docs/concepts/workloads/controllers/jobs-run-to-completion/
    * A job creates one or more pods and ensures that a specified number of them successfully terminate. As pods successfully complete, the job tracks the successful completions. When a specified number of successful completions is reached, the job itself is complete. Deleting a Job will cleanup the pods it created.
    * When a Job completes, no more Pods are created, but the Pods are not deleted either. Keeping them around allows you to still view the logs of completed pods to check for errors, warnings, or other diagnostic output. The job object also remains after it is completed so that you can view its status. It is up to the user to delete old jobs after noting their status. When you delete the job using kubectl, all the pods it created are deleted too.
    * <b>Main types of jobs</b>:
        * Non-parallel Jobs
            * normally only one pod is started, unless the pod fails.
            * job is complete as soon as Pod terminates successfully.
        * Parallel Jobs with a fixed completion count:
            * specify a non-zero positive value for .spec.completions.
            * the job is complete when there is one successful pod for each value in the range 1 to .spec.completions.
            * not implemented yet: Each pod passed a different index in the range 1 to .spec.completions.
        * Parallel Jobs with a work queue:
            * do not specify .spec.completions, default to .spec.parallelism.
            * the pods must coordinate with themselves or an external service to determine what each should work on.
            * each pod is independently capable of determining whether or not all its peers are done, thus the entire Job is done.
            * when any pod terminates with success, no new pods are created.
            * once at least one pod has terminated with success and all pods are terminated, then the job is completed with success.
            * once any pod has exited with success, no other pod should still be doing any work or writing any output. They should all be in the process of exiting.
    * <b>Job Patterns</b>: The Job object can be used to support reliable parallel execution of Pods. The Job object is not designed to support closely-communicating parallel processes, as commonly found in scientific computing. It does support parallel processing of a set of independent but related work items.
        * https://kubernetes.io/docs/concepts/workloads/controllers/jobs-run-to-completion/#job-patterns
    * <b>TTL Controller</b>: Another way to clean up finished Jobs automatically is to use a TTL mechanism provided by a TTL controller for finished resources, by specifying the .spec.ttlSecondsAfterFinished field of the Job.
        * The TTL controller provides a TTL mechanism to limit the lifetime of resource objects that have finished execution. TTL controller only handles Jobs for now, and may be expanded to handle other resources that will finish execution, such as Pods and custom resources.
    * <b>Alternatives: Bare Pods, Replication Controller, Single Job starts Controller Pod</b>
* <b>CronJob</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#cronjob-v1beta1-batch
    * https://kubernetes.io/docs/concepts/workloads/controllers/cron-jobs/
    * A Cron Job creates Jobs on a time-based schedule.
    * One CronJob object is like one line of a crontab (cron table) file. It runs a job periodically on a given schedule, written in Cron format.
    * The Cronjob is only responsible for creating Jobs that match its schedule, and the Job in turn is responsible for the management of the Pods it represents.
    
****

## Services, Load Balancing, and Networking
* <b>Services</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#service-v1-core
    * https://kubernetes.io/docs/concepts/services-networking/service/
    * A Kubernetes Service is an abstraction which defines a logical set of Pods running somewhere in your cluster and a policy by which to access them - sometimes called a micro-service. The set of Pods targeted by a Service is determined by a Label Selector
    * For Kubernetes-native applications, Kubernetes offers a simple Endpoints API that is updated whenever the set of Pods in a Service changes. 
    * For non-native applications, Kubernetes offers a virtual-IP-based bridge to Services which redirects to the backend Pods.
    * A Service in Kubernetes is a REST object, similar to a Pod. Like all of the REST objects, a Service definition can be POSTed to the apiserver to create a new instance.
    * The Service’s selector will be evaluated continuously and the results will be POSTed to an Endpoints object.
        * https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#endpoints-v1-core
    * By default the `targetPort` will be set to the same value as the `port` field. Perhaps more interesting is that targetPort can be a string, referring to the name of a port in the backend Pods.
        * `targetPort`: is the port the container accepts traffic on, `port`: is the abstracted Service port, which can be any port other pods use to access the Service
    * <b>Virtual IPs and service proxies</b>:
        * In Kubernetes v1.0, Services are a “layer 4” (TCP/UDP over IP) construct, the proxy was purely in userspace. In Kubernetes v1.1, the Ingress API was added (beta) to represent “layer 7”(HTTP) services, iptables proxy was added too, and became the default operating mode since Kubernetes v1.2. In Kubernetes v1.8.0-beta.0, ipvs proxy was added.
        * Every node in a Kubernetes cluster runs a kube-proxy. kube-proxy is responsible for implementing a form of virtual IP for Services of type other than ExternalName.
        * <b>Proxy-mode: userspace</b>
            * kube-proxy watches the Kubernetes master for the addition and removal of Service and Endpoints objects. 
            * For each Service it opens a port on the local node. Any connections to this “proxy port” will be proxied to one of the Service’s backend Pods (as reported in Endpoints). Which backend Pod to use is decided based on the SessionAffinity of the Service. 
            * Lastly, it installs iptables rules which capture traffic to the Service’s clusterIP (which is virtual) and Port and redirects that traffic to the proxy port which proxies the backend Pod. 
            * <img src="../images/devops/services-userspace-overview.svg" width="350px">
        * <b>Proxy-mode: iptables</b>
            * kube-proxy watches the Kubernetes master for the addition and removal of Service and Endpoints objects. 
            * For each Service, it installs iptables rules which capture traffic to the Service’s clusterIP (which is virtual) and Port and redirects that traffic to one of the Service’s backend sets. 
            * For each Endpoints object, it installs iptables rules which select a backend Pod.
            * <img src="../images/devops/services-iptables-overview.svg" width="350px">
            * iptables need not switch back between userspace and kernelspace, it should be faster and more reliable than the userspace proxy. 
            * the iptables proxier cannot automatically retry another Pod if the one it initially selects does not respond. 
        * <b>Proxy-mode: ipvs</b>
            * kube-proxy watches Kubernetes Services and Endpoints, calls netlink interface to create ipvs rules accordingly and syncs ipvs rules with Kubernetes Services and Endpoints periodically, to make sure ipvs status is consistent with the expectation. When Service is accessed, traffic will be redirected to one of the backend Pods.
            * <img src="../images/devops/services-ipvs-overview.svg" width="350px">
            * Ipvs is based on netfilter hook function, but uses hash table as the underlying data structure and works in the kernel space. 
    * <b>Publishing services - service types</b>
        * `ClusterIP`: Exposes the service on a cluster-internal IP. Choosing this value makes the service only reachable from within the cluster. This is the default ServiceType.
        * `NodePort`: Exposes the service on each Node’s IP at a static port (the NodePort). A ClusterIP service, to which the NodePort service will route, is automatically created. You’ll be able to contact the NodePort service, from outside the cluster, by requesting `<NodeIP>:<NodePort>`.
            * If you set the type field to NodePort, the Kubernetes master will allocate a port from a range specified by `--service-node-port-range` flag (default: 30000-32767), and each Node will proxy that port (the same port number on every Node) into your Service. That port will be reported in your Service’s `.spec.ports[*].nodePort` field.
        * `LoadBalancer`: Exposes the service externally using a cloud provider’s load balancer. NodePort and ClusterIP services, to which the external load balancer will route, are automatically created.
        * `ExternalName`: Maps the service to the contents of the externalName field (e.g. foo.bar.example.com), by returning a CNAME record with its value. No proxying of any kind is set up. This requires version 1.7 or higher of kube-dns.
* <b>DNS for Services and Pods</b>:
    * https://github.com/kubernetes/dns/blob/master/docs/specification.md
    * https://kubernetes.io/docs/concepts/services-networking/connect-applications-service/#dns
    * Kubernetes DNS schedules a DNS Pod and Service on the cluster, and configures the kubelets to tell individual containers to use the DNS Service’s IP to resolve DNS names.
    * <b>Services</b>: https://kubernetes.io/docs/concepts/services-networking/dns-pod-service/#services
    * <b>Pod</b>: https://kubernetes.io/docs/concepts/services-networking/dns-pod-service/#pods
    * <b>Pod’s DNS Policy</b>:
        * `Default`: The Pod inherits the name resolution configuration from the node that the pods run on. See related discussion for more details.
        * `ClusterFirst (default)`: Any DNS query that does not match the configured cluster domain suffix, such as "www.kubernetes.io", is forwarded to the upstream nameserver inherited from the node. Cluster administrators may have extra stub-domain and upstream DNS servers configured.
        * `ClusterFirstWithHostNet`: For Pods running with hostNetwork, you should explicitly set its DNS policy “ClusterFirstWithHostNet”.
        * `None`: A new option value introduced in Kubernetes v1.9. It allows a Pod to ignore DNS settings from the Kubernetes environment. All DNS settings are supposed to be provided using the dnsConfig field in the Pod Spec.
    * <b>Pod’s DNS Config</b>: https://kubernetes.io/docs/concepts/services-networking/dns-pod-service/#pod-s-dns-config
        * `nameservers`: a list of IP addresses that will be used as DNS servers for the Pod. There can be at most 3 IP addresses specified. When the Pod’s dnsPolicy is set to “None”, the list must contain at least one IP address, otherwise this property is optional. The servers listed will be combined to the base nameservers generated from the specified DNS policy with duplicate addresses removed.
        * `searches`: a list of DNS search domains for hostname lookup in the Pod. This property is optional. When specified, the provided list will be merged into the base search domain names generated from the chosen DNS policy. Duplicate domain names are removed. Kubernetes allows for at most 6 search domains.
        * `options`: an optional list of objects where each object may have a name property (required) and a value property (optional). The contents in this property will be merged to the options generated from the specified DNS policy. Duplicate entries are removed.
* <b>Connecting Applications with Services</b>:
    * https://kubernetes.io/docs/concepts/services-networking/connect-applications-service/
    * We give every pod its own cluster-private-IP address so you do not need to explicitly create links between pods or mapping container ports to host ports. This means that containers within a Pod can all reach each other’s ports on localhost, and all pods in a cluster can see each other without NAT. 
* <b>Ingress</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#ingress-v1beta1-extensions
    * https://kubernetes.io/docs/concepts/services-networking/ingress/
    * An ingress can be configured to give services externally-reachable URLs, load balance traffic, terminate SSL, and offer name based virtual hosting. An ingress controller is responsible for fulfilling the ingress, usually with a loadbalancer, though it may also configure your edge router or additional frontends to help handle the traffic.
        * <i>The ingress is a beta resource. You will need an ingress controller to satisfy an ingress, simply creating the resource will have no effect.</i>
    * <b>Terminology</b>:
        * `Node`: A single virtual or physical machine in a Kubernetes cluster.
        * `Cluster`: A group of nodes firewalled from the internet, that are the primary compute resources managed by Kubernetes.
        * `Edge router`: A router that enforces the firewall policy for your cluster. This could be a gateway managed by a cloud provider or a physical piece of hardware.
        * `Cluster network`: A set of links, logical or physical, that facilitate communication within a cluster according to the Kubernetes networking model.
        * `Service`: A Kubernetes Service that identifies a set of pods using label selectors. Unless mentioned otherwise, Services are assumed to have virtual IPs only routable within the cluster network.
    * <b>Ingress controllers</b>: In order for the ingress resource to work, the cluster must have an ingress controller running. This is unlike other types of controllers, which run as part of the kube-controller-manager binary, and are typically started automatically with a cluster. 
        * https://kubernetes.io/docs/concepts/services-networking/ingress/#ingress-controllers
    * <b>NetworkPolicy</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#networkpolicy-v1-networking-k8s-io
        * https://kubernetes.io/docs/concepts/services-networking/network-policies/
* Adding entries to Pod /etc/hosts with HostAliases: https://kubernetes.io/docs/concepts/services-networking/add-entries-to-pod-etc-hosts-with-host-aliases/

****

## Storage
* <b>Volumes</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#volume-v1-core
    * https://kubernetes.io/docs/concepts/storage/volumes/
    * <i>A process in a container sees a filesystem view composed from their Docker image and volumes. The Docker image is at the root of the filesystem hierarchy, and any volumes are mounted at the specified paths within the image.</i>
    * <b>Types of Volumes</b>: https://kubernetes.io/docs/concepts/storage/volumes/#types-of-volumes
        * <i>example: https://kubernetes.io/docs/concepts/storage/volumes/#hostpath; https://github.com/kubernetes/examples/tree/master/staging/volumes</i>
    * <b>subPath</b>: it is useful to share one volume for multiple uses in a single Pod. The volumeMounts.subPath property can be used to specify a sub-path inside the referenced volume instead of its root.
        * https://kubernetes.io/docs/concepts/storage/volumes/#using-subpath
    * <b>Mount propagation</b>: Mount propagation allows for sharing volumes mounted by a Container to other Containers in the same Pod, or even to other Pods on the same node.
        * Mount propagation of a volume is controlled by `mountPropagation` field in `Container.volumeMounts`. Its values are:
            * `None`: This volume mount will not receive any subsequent mounts that are mounted to this volume or any of its subdirectories by the host. This is the default mode.
            * `HostToContainer`: This volume mount will receive all subsequent mounts that are mounted to this volume or any of its subdirectories.
            * `Bidirectional`: This volume mount behaves the same the HostToContainer mount. In addition, all volume mounts created by the Container will be propagated back to the host and to all Containers of all Pods that use the same volume.
* <b>Persistent Volumes</b>: The PersistentVolume subsystem provides an API for users and administrators that abstracts details of how storage is provided from how it is consumed. 
    * https://kubernetes.io/docs/concepts/storage/persistent-volumes/
    * <b>PersistentVolume</b>: A PersistentVolume (PV) is a piece of storage in the cluster that has been provisioned by an administrator. It is a resource in the cluster just like a node is a cluster resource. 
        * https://kubernetes.io/docs/concepts/storage/persistent-volumes/#persistent-volumes
        * https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#persistentvolume-v1-core
    * <b>PersistentVolumeClaim</b>: A PersistentVolumeClaim (PVC) is a request for storage by a user. It is similar to a pod. Pods consume node resources and PVCs consume PV resources. Pods can request specific levels of resources (CPU and Memory). Claims can request specific size and access modes (e.g., can be mounted once read/write or many times read-only).
        * https://kubernetes.io/docs/concepts/storage/persistent-volumes/#persistentvolumeclaims
        * https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#persistentvolumeclaim-v1-core
    * <b>Types of Persistent Volumes</b>: https://kubernetes.io/docs/concepts/storage/persistent-volumes/#types-of-persistent-volumes
* <b>Volume Snapshots</b>: https://kubernetes.io/docs/concepts/storage/volume-snapshots/
    * `VolumeSnapshotContent` is a snapshot taken from a volume in the cluster that has been provisioned by an administrator. It is a resource in the cluster just like a PersistentVolume is a cluster resource.
        * https://kubernetes.io/docs/concepts/storage/volume-snapshots/#volume-snapshot-contents
    * `VolumeSnapshot` is a request for snapshot of a volume by a user. It is similar to a PersistentVolumeClaim.
        * https://kubernetes.io/docs/concepts/storage/volume-snapshots/#volumesnapshots
* <b>Storage Classes</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#storageclass-v1-storage-k8s-io
    * https://kubernetes.io/docs/concepts/storage/storage-classes/
    * A StorageClass provides a way for administrators to describe the “classes” of storage they offer. Different classes might map to quality-of-service levels, or to backup policies, or to arbitrary policies determined by the cluster administrators. 
    * Each StorageClass contains the fields provisioner, parameters, and reclaimPolicy, which are used when a PersistentVolume belonging to the class needs to be dynamically provisioned.
* <b>Volume Snapshot Classes</b>: 
    * https://kubernetes.io/docs/concepts/storage/volume-snapshot-classes/
    * Just like StorageClass provides a way for administrators to describe the “classes” of storage they offer when provisioning a volume, VolumeSnapshotClass provides a way to describe the “classes” of storage when provisioning a volume snapshot.
    * Each VolumeSnapshotClass contains the fields snapshotter and parameters, which are used when a VolumeSnapshot belonging to the class needs to be dynamically provisioned.
* <b>Dynamic Volume Provisioning</b>: 
    * https://kubernetes.io/docs/concepts/storage/dynamic-provisioning/
    * Dynamic volume provisioning allows storage volumes to be created on-demand. Without dynamic provisioning, cluster administrators have to manually make calls to their cloud or storage provider to create new storage volumes, and then create PersistentVolume objects to represent them in Kubernetes. The dynamic provisioning feature eliminates the need for cluster administrators to pre-provision storage. Instead, it automatically provisions storage when it is requested by users.
    * The implementation of dynamic volume provisioning is based on the API object StorageClass from the API group storage.k8s.io. A cluster administrator can define as many StorageClass objects as needed, each specifying a volume plugin (aka provisioner) that provisions a volume and the set of parameters to pass to that provisioner when provisioning. A cluster administrator can define and expose multiple flavors of storage (from the same or different storage systems) within a cluster, each with a custom set of parameters.

****

## Configuration
* <b>Configuration Best Practices</b>: https://kubernetes.io/docs/concepts/configuration/overview/
* <b>Managing Compute Resources for Containers</b>: https://kubernetes.io/docs/concepts/configuration/manage-compute-resources-container/
    * <b>Resource types</b>: CPU and memory are each a resource type. A resource type has a base unit. CPU is specified in units of cores, and memory is specified in units of bytes.
    * https://kubernetes.io/docs/concepts/configuration/manage-compute-resources-container/#meaning-of-cpu
    * https://kubernetes.io/docs/concepts/configuration/manage-compute-resources-container/#meaning-of-memory
    * <b>Resource requests and limits of Pod and Container</b>: A Pod resource request/limit for a particular resource type is the sum of the resource requests/limits of that type for each Container in the Pod.
        * Limits describes the maximum amount of compute resources allowed.
        * Requests describes the minimum amount of compute resources required.
        * `spec.containers[].resources.limits.cpu`
        * `spec.containers[].resources.limits.memory`
        * `spec.containers[].resources.requests.cpu`
        * `spec.containers[].resources.requests.memory`
    * <b>Local ephemeral storage</b>: In each Kubernetes node, kubelet’s root directory (/var/lib/kubelet by default) and log directory (/var/log) are stored on the root partition of the node. This partition is also shared and consumed by Pods via emptyDir volumes, container logs, image layers and container writable layers. This partition is “ephemeral” and applications cannot expect any performance SLAs (Disk IOPS for example) from this partition.
        * `spec.containers[].resources.limits.ephemeral-storage`
        * `spec.containers[].resources.requests.ephemeral-storage`
* <b>Assigning Pods to Nodes</b>: https://kubernetes.io/docs/concepts/configuration/assign-pod-node/
    * <b>nodeSelector</b>: nodeSelector is the simplest recommended form of node selection constraint. nodeSelector is a field of PodSpec. It specifies a map of key-value pairs. For the pod to be eligible to run on a node, the node must have each of the indicated key-value pairs as labels
    * <b>Affinity and anti-affinity</b>: https://kubernetes.io/docs/concepts/configuration/assign-pod-node/#affinity-and-anti-affinity
        * The affinity feature consists of two types of affinity, “node affinity” and “inter-pod affinity/anti-affinity”. Node affinity is like the existing nodeSelector, while inter-pod affinity/anti-affinity constrains against pod labels rather than node labels.
        * nodeSelector continues to work as usual, but will eventually be deprecated, as node affinity can express everything that nodeSelector can express.
        * <b>Node affinity</b>: https://kubernetes.io/docs/concepts/configuration/assign-pod-node/#node-affinity-beta-feature
            * Node affinity is conceptually similar to nodeSelector – it allows you to constrain which nodes your pod is eligible to be scheduled on, based on labels on the node.
            * here are currently two types of node affinity, called `requiredDuringSchedulingIgnoredDuringExecution` and `preferredDuringSchedulingIgnoredDuringExecution`. You can think of them as “hard” and “soft” respectively, in the sense that the former specifies rules that must be met for a pod to be scheduled onto a node, while the latter specifies preferences that the scheduler will try to enforce but will not guarantee.
        * <b>Inter-pod affinity and anti-affinity</b>: https://kubernetes.io/docs/concepts/configuration/assign-pod-node/#inter-pod-affinity-and-anti-affinity-beta-feature
            * Inter-pod affinity and anti-affinity allow you to constrain which nodes your pod is eligible to be scheduled based on labels on pods that are already running on the node rather than based on labels on nodes.
            * The pod affinity rule says that the pod can be scheduled onto a node only if that node is in the same zone as at least one already-running pod that has a label with key “security” and value “S1”.
            * The pod anti-affinity rule says that the pod prefers not to be scheduled onto a node if that node is already running a pod with label having key “security” and value “S2”.
    * <b>nodeName</b>: https://kubernetes.io/docs/concepts/configuration/assign-pod-node/#nodename
        * nodeName is a field of PodSpec. If it is non-empty, the scheduler ignores the pod and the kubelet running on the named node tries to run the pod. Thus, if nodeName is provided in the PodSpec, it takes precedence over the above methods for node selection.
* <b>Taints and Tolerations</b>: https://kubernetes.io/docs/concepts/configuration/taint-and-toleration/
    * Taints and tolerations work together to ensure that pods are not scheduled onto inappropriate nodes. One or more taints are applied to a node; this marks that the node should not accept any pods that do not tolerate the taints. Tolerations are applied to pods, and allow the pods to schedule onto nodes with matching taints.
* <b>Secrets</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#secret-v1-core
    * https://kubernetes.io/docs/concepts/configuration/secret/
    * <b>Using Secrets as Files from a Pod</b>: https://kubernetes.io/docs/concepts/configuration/secret/#using-secrets-as-files-from-a-pod
    * <b>Using Secrets as Environment Variables</b>: https://kubernetes.io/docs/concepts/configuration/secret/#using-secrets-as-environment-variables
* <i>Organizing Cluster Access Using kubeconfig Files</i>: https://kubernetes.io/docs/concepts/configuration/organize-cluster-access-kubeconfig/
    * Use kubeconfig files to organize information about clusters, users, namespaces, and authentication mechanisms. The kubectl command-line tool uses kubeconfig files to find the information it needs to choose a cluster and communicate with the API server of a cluster.
     * By default, kubectl looks for a file named config in the HOME/.kube directory. You can specify other kubeconfig files by setting the KUBECONFIG environment variable or by setting the --kubeconfig flag.
* <i>Pod Priority and Preemption</i>: https://kubernetes.io/docs/concepts/configuration/pod-priority-preemption/
     * Pods can have priority. Priority indicates the importance of a Pod relative to other Pods. If a Pod cannot be scheduled, the scheduler tries to preempt (evict) lower priority Pods to make scheduling of the pending Pod possible.
     
****

## Policies
* <b>Resource Quotas</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#resourcequota-v1-core
    * https://kubernetes.io/docs/concepts/policy/resource-quotas/
    * A resource quota, defined by a ResourceQuota object, provides constraints that limit aggregate resource consumption per namespace. It can limit the quantity of objects that can be created in a namespace by type, as well as the total amount of compute resources that may be consumed by resources in that project.
* <b>Pod Security Policies</b>: https://kubernetes.io/docs/reference/generated/kubernetes-api/v1.13/#podsecuritypolicy-v1beta1-extensions
    * https://kubernetes.io/docs/concepts/policy/pod-security-policy/
    * A Pod Security Policy is a cluster-level resource that controls security sensitive aspects of the pod specification. The PodSecurityPolicy objects define a set of conditions that a pod must run with in order to be accepted into the system, as well as defaults for the related fields. 

****

# 应用实战

### 应用场景
* <b>Local-machine Solutions</b>: If you just want to “kick the tires” on Kubernetes, use the local Docker-based solutions. https://kubernetes.io/docs/setup/pick-right-solution/#local-machine-solutions
* <b>Hosted Solutions</b>: When you are ready to scale up to more machines and higher availability, a hosted solution is the easiest to create and maintain. https://kubernetes.io/docs/setup/pick-right-solution/#hosted-solutions
* <b>Turnkey Cloud Solutions</b>: Turnkey cloud solutions require only a few commands to create and cover a wide range of cloud providers. https://kubernetes.io/docs/setup/pick-right-solution/#turnkey-cloud-solutions
    * <b><i>https://kubernetes.io/docs/setup/turnkey/aws/</i></b>
* <b>On-Premises turnkey cloud solutions</b>: On-Premises turnkey cloud solutions have the simplicity of the turnkey cloud solution combined with the security of your own private network.https://kubernetes.io/docs/setup/pick-right-solution/#on-premises-turnkey-cloud-solutions
* <b>Custom Solutions</b>: Custom solutions vary from step-by-step instructions to general advice for setting up a Kubernetes cluster from scratch. https://kubernetes.io/docs/setup/pick-right-solution/#custom-solutions

****

### 动手实操
* <b>Task practice and example: https://kubernetes.io/docs/tasks/</b>
* <b>Tutorials: https://kubernetes.io/docs/tutorials/</b>
* <b>Installing kubeadm</b>: https://kubernetes.io/docs/setup/independent/install-kubeadm/
    * `kubeadm`: the command to bootstrap the cluster.
    * `kubelet`: the component that runs on all of the machines in your cluster and does things like starting pods and containers.
    * `kubectl`: the command line util to talk to your cluster.
* <b>Creating a single master cluster with kubeadm</b>: https://kubernetes.io/docs/setup/independent/create-cluster-kubeadm/
* <b>Options for Highly Available Topology</b>: https://kubernetes.io/docs/setup/independent/ha-topology/
    * <b>A stacked HA cluster</b> is a topology where the distributed data storage cluster provided by etcd is stacked on top of the cluster formed by the nodes managed by kubeadm that run control plane components.
        * Creating Highly Available Clusters with kubeadm: https://kubernetes.io/docs/setup/independent/high-availability/
    * <b>An HA cluster with external etcd</b> is a topology where the distributed data storage cluster provided by etcd is external to the cluster formed by the nodes that run control plane components.
        * Set up a High Availability etcd cluster with kubeadm: https://kubernetes.io/docs/setup/independent/setup-ha-etcd-with-kubeadm/
    <table>
        <tr>
            <td><img src="../images/devops/kubeadm-ha-topology-stacked-etcd.svg" width="450px"></td>
            <td><img src="../images/devops/kubeadm-ha-topology-external-etcd.svg" width="450px"></td>
        </tr>
    </table>
* <b>Configuring each kubelet in your cluster using kubeadm</b>: https://kubernetes.io/docs/setup/independent/kubelet-integration/
* <b>Kubernetes Version and Version Skew Support Policy</b>: https://kubernetes.io/docs/setup/version-skew-policy/
* <b>Running Kubernetes Locally via Minikube</b>: https://kubernetes.io/docs/setup/minikube/
* <i>Building Large Clusters</i>: https://kubernetes.io/docs/setup/cluster-large/
* <i>Running in Multiple Zones</i>: https://kubernetes.io/docs/setup/multiple-zones/
* <i>Creating a Custom Cluster from Scratch</i>: https://kubernetes.io/docs/setup/scratch/

****

### Cluster Administration
* https://kubernetes.io/docs/concepts/cluster-administration/cluster-administration-overview/
* https://kubernetes.io/docs/concepts/extend-kubernetes/extend-cluster/
   
****

### References: https://kubernetes.io/docs/reference/
* <b>The Kubernetes API</b>:
    * API conventions doc: https://github.com/kubernetes/community/blob/master/contributors/devel/api-conventions.md
    * API Reference: https://kubernetes.io/docs/reference/
    * kubectl Commands: https://kubernetes.io/docs/reference/generated/kubectl/kubectl-commands
    * Controlling API Access doc: https://kubernetes.io/docs/reference/access-authn-authz/controlling-access/